# Import OpenTelemetry Modules for Logs

In [ ]:
import logging
import socket

from opentelemetry.sdk.resources import Resource
from opentelemetry._logs import set_logger_provider
from opentelemetry.exporter.otlp.proto.grpc._log_exporter import (
    OTLPLogExporter,
)
from opentelemetry.sdk._logs.export import (
    ConsoleLogExporter,
    BatchLogRecordProcessor,
    SimpleLogRecordProcessor
)
from opentelemetry.sdk._logs import LoggerProvider, LoggingHandler


# Create a LoggerProvider that we can send logs to

In [ ]:
logger_provider = LoggerProvider(
    resource=Resource.create(
        {
            "service.name": "section02",
            "deployment.environment": "otel",
            "host.name": socket.gethostname(),
        }
    ),
)
set_logger_provider(logger_provider)

# Create log events and send them to the collector

In [ ]:
logging.basicConfig(level=logging.NOTSET)

log_exporter = OTLPLogExporter(endpoint="localhost:4317", insecure=True)
logger_provider.add_log_record_processor(SimpleLogRecordProcessor(log_exporter))
# logger_provider.add_log_record_processor(BatchLogRecordProcessor(log_exporter))
# logger_provider.add_log_record_processor(BatchLogRecordProcessor(ConsoleLogExporter()))
logger_provider.add_log_record_processor(SimpleLogRecordProcessor(ConsoleLogExporter()))

handler = LoggingHandler(level=logging.DEBUG, logger_provider=logger_provider)
# handler = LoggingHandler(level=logging.INFO, logger_provider=logger_provider)

# Attach OTLP handler to root logger
logging.getLogger().addHandler(handler)

logger1 = logging.getLogger(__name__)

In [ ]:
logger1.debug("This is a debug message.")
logger1.info("This is an info message.")
# logger1.warning("This is a warning message.")
# logger1.error("This is an error message.")

### View output

View the results in Datadog: https://app.datadoghq.com/logs/livetail?query=%40service.name%3Asection02


# Shutdown the logger when complete

In [ ]:
logger_provider.shutdown()